In [1]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/PL

/content/drive/MyDrive/PL


In [3]:
# !pip install cmath
!pip install torch
!pip install sklearn
!pip install pandas
!pip install librosa
!pip install numpy
!pip install speechpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=2bab7c077ef4718501b66322edc55ce7dcb98bfabcc951a9be7881769637c22a
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 31.3 MB/s 
     |████████████████████████████████| 182 kB 77.6 MB/s 
     |████████████████████████████████| 7.6 MB 75.7 MB/s 


In [5]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/PL/metadata_train.csv")
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)
train.to_csv("train.csv")
test.to_csv("test.csv")

In [12]:
from cmath import acos
import torch
from torch.utils.data import Dataset
# import matplotlib.pyplot as plt
import os
import pandas as pd
# import numpy as np
# from torch.utils.data import DataLoader
# from infer import phonetic_embedding
from help import Atention, wav_norm
from char_embedding import tensor_to_text,text_to_tensor
import librosa
import numpy as np

data = pd.read_csv("train.csv")
# print(data['Wav_path'])
sample = data.shape[0]
# cols = ['Wav_path', 'Lyric_path']

class MDD_Dataset(Dataset):

    def __init__(self):
        acoustic_canonical = data
        self.n_samples = sample
        A = acoustic_canonical['Wav_path']
        C = acoustic_canonical['Lyric_path']
        B = acoustic_canonical['Lyric_path'] #output
        

        self.A_data = A 
        self.C_data = C
        self.y_data = B 

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        p = self.A_data[index]
        p = p.split(".")[0]
        base_dir = '/content/drive/MyDrive/raw_song_phonetic/'
        p = p + str(".npy")
        p = np.load(base_dir + str(p))
        phonetic = torch.tensor(p)
       
        linguistic = text_to_tensor(self.C_data[index])
        linguistic = torch.tensor(linguistic)
        label = text_to_tensor(self.y_data[index])
        label = torch.tensor(label)
        return phonetic, linguistic, label

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples



In [ ]:
# from model import Net
# from cProfile import label
# from infer import phonetic_embedding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torch.nn as nn
import torch.nn.functional as F
from phonetic_encoder import Phonetic_encoder
from acoustic_encoder import Acoustic_encoder
from linguistic_encoder import Linguistic_encoder
from char_embedding import text_to_tensor
from help import wav_norm, Atention
import numpy as np
# from help import beam_search_decoding
from model import Acoustic_Phonetic_Linguistic
# from dataloader import MDD_Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
tokenizer = Wav2Vec2Processor.from_pretrained("pretrained_finetuned")

"""
"""
f = open("loss.txt", 'a')
MDD = MDD_Dataset()
# print(data)
net = Acoustic_Phonetic_Linguistic()
net.to('cuda')

# net = torch.load('/home/tuht/train_wav2vec/MDD_Checkpoint/checkpoint_AdamW_16head_PL.pth')
# net = net.to('cpu')
train_loader = DataLoader(dataset=MDD,
                          batch_size=1,
                          shuffle=True,
                          num_workers=0)



# convert to an iterator and look at one random sample


ctc_loss = nn.CTCLoss(blank = 95)
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
optimizer = optim.AdamW(net.parameters(), lr = 0.00001)
# optimizer = optim.SGD(net.parameters(), 0.01, momentum = 0.9)
# optimizer = torch.load('/home/tuht/train_wav2vec/MDD_Checkpoint/checkpoint_optim.pth')
for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        phonetic, linguistic, labels = data
        # acoustic = torch.tensor(0).to('cuda')
        phonetic = phonetic.to('cuda')
        linguistic = linguistic.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(phonetic, linguistic)
        outputs = outputs.unsqueeze(1)
        input_lengths = outputs.shape
        target_lengths = labels.shape
        target = labels
        input_lengths = [input_lengths[0]]
        target_lengths =[target_lengths[1]]
        input_lengths = torch.tensor(input_lengths)
        target_lengths = torch.tensor(target_lengths)
        outputs = (F.log_softmax(outputs, dim=2))
        loss = ctc_loss(outputs, labels, input_lengths, target_lengths)
        print(i)
        print(loss)
        f.write("(" +str(epoch) + "," + str(i) + ")" + "  loss: " + str(loss) + "\n") 
        loss.backward()
        optimizer.step()
    torch.save(net, 'raw_checkpoint.pth' + str(epoch))
        
print('Finished Training')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/content/drive/MyDrive/PL/linguistic_encoder.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float)


Streaming output truncated to the last 5000 lines.
375
tensor(1.0694, device='cuda:0', grad_fn=<MeanBackward0>)
376
tensor(2.4930, device='cuda:0', grad_fn=<MeanBackward0>)
377
tensor(2.5916, device='cuda:0', grad_fn=<MeanBackward0>)
378
tensor(1.6097, device='cuda:0', grad_fn=<MeanBackward0>)
379
tensor(1.3908, device='cuda:0', grad_fn=<MeanBackward0>)
380
tensor(0.8994, device='cuda:0', grad_fn=<MeanBackward0>)
381
tensor(1.4033, device='cuda:0', grad_fn=<MeanBackward0>)
382
tensor(1.7684, device='cuda:0', grad_fn=<MeanBackward0>)
383
tensor(2.7268, device='cuda:0', grad_fn=<MeanBackward0>)
384
tensor(1.3177, device='cuda:0', grad_fn=<MeanBackward0>)
385
tensor(1.2931, device='cuda:0', grad_fn=<MeanBackward0>)
386
tensor(1.5045, device='cuda:0', grad_fn=<MeanBackward0>)
387
tensor(0.7724, device='cuda:0', grad_fn=<MeanBackward0>)
388
tensor(1.1095, device='cuda:0', grad_fn=<MeanBackward0>)
389
tensor(1.8363, device='cuda:0', grad_fn=<MeanBackward0>)
390
tensor(1.1113, device='cuda:0'